In [4]:
import numpy as np

#Eigendecomposition is used. U = QAQ^(-1)
#where A is a diagonal matrix whose entries are the eigenvalues of U,
#and Q is a matrix whose columns are the eigenvectors of U

#Procedure goes as follows:
#1) Generate random Q matrix
#2) Check is invertible. If not generate new random Q matrix
#3) Assuming Q is invertible, generate diagonales entries of A 
#   such that they have the needed properties (e.g. eigenvalues>0)
#4) Calculate U


#Parameters for Q
N = 4      #dimension of random square matrix
scale = 1 #random matrix by fault between [0,1), reason why it should be scaled

#Generate nonsingunlar Q (invertible)
valid = False
while True:
    Q     = scale*np.random.rand(N,N)
    try:
        inv_Q = np.linalg.inv(Q)
        break
    except np.linalg.LinAlgError:
        print("sampled Q matrix is not invertible")


#Generate eigenvalues for A
lb = -1 #lower bound for eigenvalues
ub = 1  #upper bound for eigenvalues
eigval = (ub-lb)*np.random.random(N) + lb #numpy function samples from U(0,1)
A = np.diag(eigval)

#Calculate desired matrix U and verify correct eigenvalues
U = Q @ A @ inv_Q#np.linalg.inv(Q)
print(eigval)
print(np.linalg.eigvals(U))

[ 0.27597967 -0.16096451  0.34053674 -0.62989491]
[-0.62989491 -0.16096451  0.34053674  0.27597967]


# Trabajo pendiente
1) Programar maximum likelihood (optimizacion)
2) Calcular periodic trend